<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/nbs/models/bilstm-bigru-1epochs-2lac-prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 8.1MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
     |████████████████████████████████| 3.0MB 50.3MB/s 
     |████████████████████████████████| 1.1MB 34.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9c3f5cc088c18add25ac48aa9f9c6de6afea326e02dd8542952f0f6b39e5b1d1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import TFAutoModelWithLMHead, AutoTokenizer
from transformers import pipeline
from pprint import pprint
from transformers import *


@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "roberta-base" # huggingface bert model 


flags = Config()
AUTO = tf.data.experimental.AUTOTUNE


"""
## Set-up TPU Runtime
"""


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
!unzip ./Embold_Participant\'s_Dataset.zip -d Dataset

--2020-10-10 05:04:11--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.66.56
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.66.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  12.6MB/s    in 9.5s    

2020-10-10 05:04:22 (10.3 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Dataset/Embold_Participant's_Dataset/
  inflating: Dataset/Embold_Participant's_Dataset/sample submission.csv  
  inflating: Dataset/__MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Dataset/Embold_Participant's_Dataset/embold_train_extra

In [4]:
cd "Dataset/Embold_Participant's_Dataset/"

/content/Dataset/Embold_Participant's_Dataset


In [5]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
train_ex_df = pd.read_json("embold_train_extra.json")
train_data = train_df.append(train_ex_df)
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

data = train_data[['text']].append(test_df[['text']])

In [7]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

from transformers import *
#tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
tokenizer = RobertaTokenizerFast.from_pretrained(flags.PRETRAINED_MODEL)
#tokenizer = RobertaTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

In [ ]:
%%time

X_data = regular_encode(train_data.text.values.tolist()[200000:400000], tokenizer, maxlen=flags.MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
y_train = train_data.label.values

In [16]:
tokenizer.vocab_size

50265

In [21]:
tf.keras.backend.clear_session()

In [22]:
with strategy.scope():
  input_layer = tf.keras.layers.Input((flags.MAX_LEN,), dtype=tf.int32)
  embeddings = tf.keras.layers.Embedding(tokenizer.vocab_size, 128)(input_layer)
  embeddings = tf.keras.layers.SpatialDropout1D(0.3)(embeddings)
  gru_layer = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True))(embeddings)
  lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(gru_layer)
  pooled_output = tf.keras.layers.GlobalMaxPooling1D()(lstm_layer)
  output = tf.keras.layers.Dense(3, activation='softmax')(pooled_output)
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam()
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 320, 128)          6433920   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 320, 128)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 320, 512)          592896    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 320, 256)          656384    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                

In [12]:
global_batch_size

16

In [23]:
classifier_model.fit(X_data, y_train[0:200000], epochs=1, batch_size=128, validation_split=0.1)

1407/1407 [==============================] - 322s 229ms/step - loss: 0.5447 - accuracy: 0.7866 - val_loss: 0.4990 - val_accuracy: 0.8084


In [ ]:
classifier_model.fit(X_data, y_train[200000:400000], epochs=1, batch_size=128, validation_split=0.1)

## Predict and Prepare for submission

In [25]:
%%time

X_test_data = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=flags.MAX_LEN)
X_test_data.shape

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


CPU times: user 13.9 s, sys: 271 ms, total: 14.1 s
Wall time: 7.99 s


In [26]:
y_test_pred = classifier_model.predict(X_test_data, batch_size=128, verbose=1)

235/235 [==============================] - 15s 63ms/step


In [27]:
y_test_pred_label = y_test_pred.argmax(axis=1)

In [28]:
submission = pd.read_csv('sample submission.csv')

In [29]:
submission['label'] = y_test_pred_label
submission.to_csv('bilstm-bigru-1epochs-2lac.csv', index=False)

In [30]:
submission.head()

,label
0,2
1,0
2,0
3,0
4,1


In [31]:
submission['label'] = y_test_pred.tolist()
submission.to_csv('bilstm-bigru-1epochs-2lac-prob.csv', index=False)

In [32]:
submission.head()

,label
0,"[0.03425627574324608, 0.09429800510406494, 0.8..."
1,"[0.9079707264900208, 0.08556487411260605, 0.00..."
2,"[0.887165904045105, 0.06275520473718643, 0.050..."
3,"[0.5345097780227661, 0.19793424010276794, 0.26..."
4,"[0.03544525429606438, 0.9540987610816956, 0.01..."
